# Indexing and selecting data

In [5]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## More on NumPy indexing

In [16]:
a = np.array([-2, 3, 4, -5, 5])
print(a)

[-2  3  4 -5  5]


### Fancy indexing

Apart from indexing with integers and slices NumPy also supports indexing with arrays of integers (so-called *fancy indexing*). For example, to get the 2nd and 4th element of ``a``:

In [7]:
a[[1, 3]]

array([ 3, -5])

### Boolean indexing

To select data fulfilling specific criteria, one can use the *bolean indexing*. This is best illustrated on 1D arrays; for example, lets select only positive elements of ``a``:


In [8]:
a[a > 0]

array([3, 4, 5])

Note that the index array has the same size as and type of boolean:

In [9]:
print(a)
print(a > 0)

[-2  3  4 -5  5]
[False  True  True False  True]


Multiple criteria can be also combine in one query:

In [10]:
a[(a > 0) & (a < 5)]

array([3, 4])

<div class="alert alert-success">
    <b>EXERCISE</b>: Select all odd numbers from the array <code>a</code>
</div>

In [19]:

a = np.array([-2, 3, 4, -5, 5])
print (a[(a+1)%2==0])

[ 3 -5  5]


<div class="alert alert-success">
    <b>EXERCISE</b>: Select <b>negative</b> odd numbers from the array <code>a</code>
</div>

In [20]:
a = np.array([-2, 3, 4, -5, 5])
print (a[((a+1)%2==0)&(a<0)])

[-5]


## Indexing pandas `Series`

``Series`` can be indexed similarly to 1D NumPy array. 

In [21]:
pop_dict = {'Germany': 81.3, 
            'Belgium': 11.3, 
            'France': 64.3, 
            'United Kingdom': 64.9, 
            'Netherlands': 16.9}
population = pd.Series(pop_dict)
print(population)

Belgium           11.3
France            64.3
Germany           81.3
Netherlands       16.9
United Kingdom    64.9
dtype: float64


We can use fancy indexing with the rich index:

In [22]:
population[['Netherlands', 'Germany']]

Netherlands    16.9
Germany        81.3
dtype: float64

Similarly, boolean indexing can be used to filter the ``Series``. Lets select countries with population of more than 20 millions:

In [23]:
population[population > 20]

France            64.3
Germany           81.3
United Kingdom    64.9
dtype: float64

You can also do position-based indexing by using integers instead of labels:

In [24]:
population[:2]

Belgium    11.3
France     64.3
dtype: float64

## Indexing `DataFrame`

In [88]:
data = {'country': ['Belgium', 'France', 'Germany', 'Netherlands', 'United Kingdom'],
        'population': [11.3, 64.3, 81.3, 16.9, 64.9],
        'area': [30510, 671308, 357050, 41526, 244820],
        'capital': ['Brussels', 'Paris', 'Berlin', 'Amsterdam', 'London']}
countries = pd.DataFrame(data)
countries

,area,capital,country,population
0,30510,Brussels,Belgium,11.3
1,671308,Paris,France,64.3
2,357050,Berlin,Germany,81.3
3,41526,Amsterdam,Netherlands,16.9
4,244820,London,United Kingdom,64.9


In [89]:
countries = countries.set_index('country')
countries

,area,capital,population
country,,,
Belgium,30510,Brussels,11.3
France,671308,Paris,64.3
Germany,357050,Berlin,81.3
Netherlands,41526,Amsterdam,16.9
United Kingdom,244820,London,64.9


## Some notes on selecting data

Data frames allow for labeling rows and columns, but this makes indexing also a bit more complex compared to 1D NumPy's ``array`` and pandas ``Series``. We now have to distuinguish between:

- selection of rows or columns,
- selection by label or position.

### `[]` provides some convenience shortcuts 

For a ``DataFrame``, basic indexing selects the columns.

Selecting a single column:

In [15]:
countries['area']

country
Belgium            30510
France            671308
Germany           357050
Netherlands        41526
United Kingdom    244820
Name: area, dtype: int64

or multiple columns using fancy indexing:

In [16]:
countries[['area', 'population']]

,area,population
country,,
Belgium,30510,11.3
France,671308,64.3
Germany,357050,81.3
Netherlands,41526,16.9
United Kingdom,244820,64.9


But, slicing accesses the rows:

In [17]:
countries['France':'Netherlands']

,area,capital,population
country,,,
France,671308,Paris,64.3
Germany,357050,Berlin,81.3
Netherlands,41526,Amsterdam,16.9


We can also select rows similarly to the boolean indexing in numpy. The boolean mask should be 1-dimensional and the same length as the thing being indexed. Boolean indexing of `DataFrame`  can be used like the `WHERE` clause of SQL to select **rows** matching some criteria:

In [18]:
countries[countries['area'] > 100000]

,area,capital,population
country,,,
France,671308,Paris,64.3
Germany,357050,Berlin,81.3
United Kingdom,244820,London,64.9


So as a summary, `[]` provides the following convenience shortcuts:

<table>
<tr>
<td></td>
<td>NumPy/`Series`</td>
<td>`DataFrame`</td>
</tr>
<tr>
<td>Integer index<br>`data[label]`</td>
<td>single element</td>
<td>single **column**</td>
</tr>
<tr>
<td>Slice<br>`data[label1:label2]`</td>
<td>sequence</td>
<td>one or more **rows**</td>
</tr>
<tr>
<td>Fancy indexing<br>`data[[label1,label2]]`</td>
<td>sequence</td>
<td>one or more **columns**</td>
</tr>
<tr>
<td>Boolean indexing<br>`data[mask]`</td>
<td>sequence</td>
<td>one or more **rows**</td>
</tr>
</table>

<div class="alert alert-success">
    <b>EXERCISE</b>: Calculate the area of Germany relative to the total area of all other countries in the data frame. *Hint*: you can compare the index of the data frame to any string
</div>

In [68]:
relative_area=countries['area']['Germany']/sum(countries[countries.index!='Germany']['area'])
print (relative_area)

0.361326662376


### Systematic indexing with `loc` and `iloc`

When using `[]` like above, you can only select from one axis at once (rows or columns, not both). For more advanced indexing, you have some extra attributes:
    
* `loc`: selection by label
* `iloc`: selection by position

These methods index the different dimensions of the frame:

* `df.loc[row_indexer, column_indexer]`
* `df.iloc[row_indexer, column_indexer]`

Selecting a single element:

In [69]:
countries.loc['Germany', 'area']

357050

But the row or column indexer can also be a list, slice, boolean array, ..

In [70]:
countries.loc['France':'Germany', ['area', 'population']]

,area,population
country,,
France,671308,64.3
Germany,357050,81.3


---
Selecting by position with `iloc` works similar as indexing numpy arrays:

In [71]:
countries.iloc[:2,1:3]

,capital,population
country,,
Belgium,Brussels,11.3
France,Paris,64.3


The different indexing methods can also be used to assign data:

In [90]:
countries2 = countries.copy()
countries2.loc['Belgium':'Germany', 'population'] = 10

In [91]:
countries2

,area,capital,population
country,,,
Belgium,30510,Brussels,10.0
France,671308,Paris,10.0
Germany,357050,Berlin,10.0
Netherlands,41526,Amsterdam,16.9
United Kingdom,244820,London,64.9


---

<div class="alert alert-success">
    <b>EXERCISE</b>: Add a column `density` with the population density (note: population column is expressed in millions)
</div>

In [92]:
countries2['density']=countries2['population']*10**6/countries['area']
countries2

,area,capital,population,density
country,,,,
Belgium,30510,Brussels,10.0,327.761390
France,671308,Paris,10.0,14.896292
Germany,357050,Berlin,10.0,28.007282
Netherlands,41526,Amsterdam,16.9,406.973944
United Kingdom,244820,London,64.9,265.092721


<div class="alert alert-success">
    <b>EXERCISE</b>: Select the capital and the population column of those countries where the density is larger than 300
</div>

In [93]:
selection=countries2.loc[countries2['density']>300,['capital','population']]
selection

,capital,population
country,,
Belgium,Brussels,10.0
Netherlands,Amsterdam,16.9


<div class="alert alert-success">
    <b>EXERCISE</b>: List names, capitals and population densities of two countries with highest population density.
</div>

In [97]:
countries2=countries2.sort_values(by='density',ascending=False)
twohighest=countries2.iloc[:2,1:3]
twohighest

,capital,population
country,,
Netherlands,Amsterdam,16.9
Belgium,Brussels,10.0


<div class="alert alert-success">
    <b>EXERCISE</b>: Change the capital of the UK to Cambridge
</div>

In [99]:
countries3=countries2.copy()
countries3.loc['United Kingdom','capital']='Cambridge'
countries3

,area,capital,population,density
country,,,,
Netherlands,41526,Amsterdam,16.9,406.973944
Belgium,30510,Brussels,10.0,327.761390
United Kingdom,244820,Cambridge,64.9,265.092721
Germany,357050,Berlin,10.0,28.007282
France,671308,Paris,10.0,14.896292


<div class="alert alert-success">
    <b>EXERCISE</b>: Select all countries whose population density is between 100 and 300 people/km²
</div>

In [102]:
selection4=countries2[(countries2['density']<300)&(countries2['density']>100)].index
selection4

Index(['United Kingdom'], dtype='object', name='country')

## More exercises!

For the quick ones among you, here are some more exercises with some larger dataframe with film data. These exercises are based on the [PyCon tutorial of Brandon Rhodes](https://github.com/brandon-rhodes/pycon-pandas-tutorial/) (so all credit to him!) and the datasets he prepared for that. You can download these data from here: [`titles.csv`](https://drive.google.com/file/d/0B3G70MlBnCgKa0U4WFdWdGdVOFU/view?usp=sharing) and [`cast.csv`](https://drive.google.com/file/d/0B3G70MlBnCgKRzRmTWdQTUdjNnM/view?usp=sharing) and put them in the `/data` folder.

In [104]:
cast = pd.read_csv('data/cast.csv')
cast.head()

,title,year,name,type,character,n
0,Suuri illusioni,1985,Homo $,actor,Guests,22
1,Gangsta Rap: The Glockumentary,2007,Too $hort,actor,Himself,NaN
2,Menace II Society,1993,Too $hort,actor,Lew-Loc,27
3,Porndogs: The Adventures of Sadie,2009,Too $hort,actor,Bosco,3
4,Stop Pepper Palmer,2014,Too $hort,actor,Himself,NaN


In [105]:
titles = pd.read_csv('data/titles.csv')
titles.head()

,title,year
0,The Rising Son,1990
1,Ashes of Kukulcan,2016
2,The Thousand Plane Raid,1969
3,Crucea de piatra,1993
4,The 86,2015


<div class="alert alert-success">
    <b>EXERCISE</b>: How many movies are listed in the titles dataframe?
</div>

In [106]:
len(titles.index)

215981

<div class="alert alert-success">
    <b>EXERCISE</b>: What are the earliest two films listed in the titles dataframe?
</div>

In [113]:
titles=titles.sort_values(by='year',ascending=True)
answer=titles.iloc[:2]
answer

,title,year
165182,Miss Jerry,1894
85708,Reproduction of the Corbett and Fitzsimmons Fight,1897


<div class="alert alert-success">
    <b>EXERCISE</b>: How many movies have the title "Hamlet"?
</div>

In [115]:
hamlet=titles[titles['title']=='Hamlet']
len(hamlet)

19

<div class="alert alert-success">
    <b>EXERCISE</b>: List all of the "Treasure Island" movies from earliest to most recent.
</div>

In [117]:
treasure=titles[titles['title']=='Treasure Island']
treasure=treasure.sort_values(by='year',ascending=True)
treasure

,title,year
191379,Treasure Island,1918
47769,Treasure Island,1920
192917,Treasure Island,1934
90175,Treasure Island,1950
104714,Treasure Island,1972
103646,Treasure Island,1973
190792,Treasure Island,1985
166675,Treasure Island,1999


<div class="alert alert-success">
    <b>EXERCISE</b>: How many movies were made from 1950 through 1959?
</div>

In [118]:
titles_set=titles[(titles['year']>=1950)&(titles['year']<=1959)]
len(titles_set)

12120

<div class="alert alert-success">
    <b>EXERCISE</b>: How many roles in the movie "Inception" are NOT ranked by an "n" value?
</div>

In [139]:
inception=cast[cast['title']=='Inception']
inception_cast=inception[inception['n']>=0]
len(inception_cast)

51

<div class="alert alert-success">
    <b>EXERCISE</b>: But how many roles in the movie "Inception" did receive an "n" value?
</div>

In [140]:
len(inception)-len(inception_cast)

22

<div class="alert alert-success">
    <b>EXERCISE</b>: Display the cast of "North by Northwest" in their correct "n"-value order, ignoring roles that did not earn a numeric "n" value.
</div>

In [142]:
nowe=cast[cast['title']=='North by Northwest']
nowe=nowe[nowe['n']>=0].sort_values(by='n',ascending=True)
nowe

,title,year,name,type,character,n
778278,North by Northwest,1959,Cary Grant,actor,Roger O. Thornhill,1
3104905,North by Northwest,1959,Eva Marie Saint,actress,Eve Kendall,2
1300582,North by Northwest,1959,James Mason,actor,Phillip Vandamm,3
2794420,North by Northwest,1959,Jessie Royce Landis,actress,Clara Thornhill,4
317310,North by Northwest,1959,Leo G. Carroll,actor,The Professor,5
2702399,North by Northwest,1959,Josephine Hutchinson,actress,Mrs. Townsend,6
1513974,North by Northwest,1959,Philip Ober,actor,Lester Townsend,7
1137555,North by Northwest,1959,Martin Landau,actor,Leonard,8
2181363,North by Northwest,1959,Adam Williams,actor,Valerian,9
1616877,North by Northwest,1959,Edward Platt,actor,Victor Larrabee,10


<div class="alert alert-success">
    <b>EXERCISE</b>: How many roles were credited in the silent 1921 version of Hamlet?
</div>

In [150]:
len(cast[(cast['title']=='Hamlet')&(cast["year"]==1921)])

9

<div class="alert alert-success">
    <b>EXERCISE</b>: List the supporting roles (having n=2) played by Cary Grant in the 1940s, in order by year.
</div>

In [147]:
cary=cary[(cast['year']>=1940)&(cast['year']<=1949)&(cast['name']=='Cary Grant')].sort_values(by='year',ascending=True)
cary

/home/quentin/anaconda3/lib/python3.5/site-packages/pandas/core/frame.py:1942: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


,title,year,name,type,character,n
778302,The Philadelphia Story,1940,Cary Grant,actor,C. K. Dexter Haven,1
778300,The Howards of Virginia,1940,Cary Grant,actor,Matt Howard,1
778257,His Girl Friday,1940,Cary Grant,actor,Walter Burns,1
778275,My Favorite Wife,1940,Cary Grant,actor,Nick,2
778290,Suspicion,1941,Cary Grant,actor,Johnnie,1
778285,Penny Serenade,1941,Cary Grant,actor,Roger Adams,2
778304,The Talk of the Town,1942,Cary Grant,actor,Leopold Dilg,1
778281,Once Upon a Honeymoon,1942,Cary Grant,actor,Patrick 'Pat' O'Toole,1
778248,Destination Tokyo,1943,Cary Grant,actor,Capt. Cassidy,1
778273,Mr. Lucky,1943,Cary Grant,actor,Joe Adams,1
